In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime, timedelta

In [19]:
# ML Libraries
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [20]:
# Time Series Libraries
from prophet import Prophet

In [21]:
# Importing csv files with reading the date columns and parsing them
df1 = pd.read_csv("fct_order.csv", encoding="cp1252", parse_dates=['order_time'], date_format='%d/%m/%Y %H:%M')
df2 = pd.read_csv("dim_vendor.csv", encoding="cp1252")
df3 = pd.read_csv("dim_date.csv", encoding="cp1252", parse_dates=['iso_date'], date_format="%d-%b-%y")

In [22]:
print(df1.info())
print(df2.info())
print(df3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119885 entries, 0 to 119884
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   order_id                    119885 non-null  int64         
 1   order_time                  119885 non-null  datetime64[ns]
 2   vendor_id                   119885 non-null  int64         
 3   analytical_customer_id      119885 non-null  object        
 4   is_acquisition              119885 non-null  bool          
 5   is_successful               119885 non-null  bool          
 6   reason                      19716 non-null   object        
 7   sub_reason                  8348 non-null    object        
 8   owner                       19716 non-null   object        
 9   delivery_arrangement        119885 non-null  object        
 10  gmv_amount_lc               119885 non-null  float64       
 11  basket_amount_lc            119885 non-

In [23]:
df1.columns

Index(['order_id', 'order_time', 'vendor_id', 'analytical_customer_id',
       'is_acquisition', 'is_successful', 'reason', 'sub_reason', 'owner',
       'delivery_arrangement', 'gmv_amount_lc', 'basket_amount_lc',
       'delivery_fee_amount_lc', 'Payment_Method', 'actual_delivery_time',
       'promised_delivery_time', 'order_delay', 'dropoff_distance_manhattan',
       'platform', 'vertical_class', 'vertical', 'is_affordable_freedelivery',
       'is_affordable_item', 'is_affordable_gem', 'is_affordable_restaurant',
       'is_affordable_voucher', 'is_affordable', 'affordability_amt_total',
       'City_Cluster', 'City', 'Zone'],
      dtype='object')

In [24]:
df1.head()

,order_id,order_time,vendor_id,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,...,is_affordable_freedelivery,is_affordable_item,is_affordable_gem,is_affordable_restaurant,is_affordable_voucher,is_affordable,affordability_amt_total,City_Cluster,City,Zone
0,544145594,2021-09-01 10:41:00,609508,6zWSp9yqnEQuq4N6ykRgbbdQ,False,True,NaN,NaN,NaN,TGO,...,False,False,False,False,False,False,NaN,Cairo,Cairo,Mohandiseen
1,544478492,2021-09-01 18:17:00,602440,TtmFFw26sQR26nBE2RykhxQA,False,True,NaN,NaN,NaN,TGO,...,False,False,False,False,False,False,NaN,Cairo,Cairo,Cairo_city_centre
2,544358032,2021-09-01 15:56:00,709071,7pB4MC2aBPRdajLH2wPvIvAA,False,True,NaN,NaN,NaN,TGO,...,False,False,False,False,False,False,NaN,Cairo,Cairo,Tagamoa 5 east
3,544366924,2021-09-01 16:09:00,607670,AFedsf0e-lSZer_X0eR4VE-g,False,True,NaN,NaN,NaN,TGO,...,False,False,False,False,False,False,NaN,Cairo,Cairo,Cairo_city_centre
4,544372362,2021-09-01 16:16:00,616421,JDYqF36aULSEaWsz604DuuRw,False,True,NaN,NaN,NaN,TGO,...,False,False,False,False,False,False,NaN,Cairo,Cairo,Maadi


In [25]:
df2.head()

,vendor_id,name_en,chain_name_en,vertical_class,vertical,main_cuisine
0,738143,"Nutella Waffel 2,Faisal - Kom Bakar",Nutella Waffel 2,food,food,Waffles
1,742299,"Waffle Maker, Dokki - Mohy El Din Abou El Ezz",Waffle Maker,food,food,Waffles
2,740938,"Koshary Faisal,Giza",Koshary Faisal,food,food,Koshary
3,751673,"Koshary w Tawajin Moamen, Al Bahr Al Azam",Koshary w Tawajin Moamen,food,food,Koshary
4,763259,"Koshary Helmeya Hend, Imbabah - Bashtil",Koshary Halwani Hend,food,food,Koshary


In [26]:
df3.head()

,iso_date,year,year_quarter,year_month_str,Talabat Week,Weekday,Month Q,day_type_tlb
0,2022-01-02,2022,2022-1,2022-01,02-Jan-22,Sun,1,Weekday
1,2022-01-01,2022,2022-1,2022-01,26-Dec-21,Sat,1,Weekend
2,2021-12-31,2021,2021-4,2021-12,26-Dec-21,Fri,4,Weekend
3,2021-12-30,2021,2021-4,2021-12,26-Dec-21,Thu,4,Weekday
4,2021-12-29,2021,2021-4,2021-12,26-Dec-21,Wed,4,Weekday


In [27]:
# extracting date from order time from the fact table
df1["order_date"] = df1["order_time"].dt.strftime('%d/%m/%Y')
df3["iso_date"] = pd.to_datetime(df3["iso_date"], format="%d%m%Y")

In [28]:
# Filter successful orders and count daily 
daily_orders = df1[(df1["is_successful"] == True)].groupby("order_date").size().reset_index(name="orders")
daily_orders

,order_date,orders
0,01/01/2022,769
1,01/09/2021,819
2,01/10/2021,837
3,01/11/2021,765
4,01/12/2021,764
...,...,...
119,30/10/2021,843
120,30/11/2021,795
121,30/12/2021,780
122,31/10/2021,857


In [29]:
# Full-Range Date Index from Sep 2021 to Jan 2022
range = pd.date_range(start="2021-09-01", end="2022-01-02", freq="d")
daily_orders = daily_orders.reindex()

In [30]:
px.line(data_frame=daily_orders, x="order_date", y="orders")

In [31]:
# testing this on our dataset
test_daily_orders = df1[(df1["is_successful"] == True)].groupby("order_date").size().reset_index(name="orders")

In [32]:
# converting columns to datetime
daily_orders["order_date"] = pd.to_datetime(daily_orders["order_date"], format='%d/%m/%Y')
daily_orders = daily_orders.sort_values('order_date')

In [33]:
# Adding a feature (as documented in the excel that the talabt day starts from sunday and ends on saturday )
daily_orders["talabat_week"] = (daily_orders["order_date"].dt.day_of_week + 1) % 7

In [34]:
# preprare train and test split
# Use 75% for training (1st of Sep to 2nd of December) and test the rest (from 3rd of December to 2nd of Jan)
train_end = '2021-12-02'
test_start = '2021-12-03'
train_data = daily_orders[daily_orders["order_date"] <= train_end]
test_data = daily_orders[(daily_orders["order_date"] >= test_start) & (daily_orders["order_date"] <= '2021-01-02')]

In [35]:
# Firstly, we'll test on Nov and Dec
test_cut = '2021-11-01'
train = train_data[train_data["order_date"] < test_cut]
test = train_data[train_data["order_date"] >= test_cut]

In [36]:
train_data["order_year"] =train_data["order_date"].dt.year 
train_data["order_month"] =train_data["order_date"].dt.month
train_data["order_day"] =train_data["order_date"].dt.day

C:\Users\ezzel\AppData\Local\Temp\ipykernel_20616\4190417397.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ezzel\AppData\Local\Temp\ipykernel_20616\4190417397.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ezzel\AppData\Local\Temp\ipykernel_20616\4190417397.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [37]:
train_data = train_data.drop('order_date', axis=1)
train_data

,orders,talabat_week,order_year,order_month,order_day
1,819,3,2021,9,1
6,856,4,2021,9,2
10,837,5,2021,9,3
14,845,6,2021,9,4
18,847,0,2021,9,5
...,...,...,...,...,...
112,779,0,2021,11,28
116,719,1,2021,11,29
120,795,2,2021,11,30
4,764,3,2021,12,1


In [38]:
xx = train_data[["order_year", "order_month", "order_day", "talabat_week"]]
yy = train_data[["orders"]]

In [39]:
train_x, test_x, train_y, test_y = train_test_split(xx, yy, test_size=0.25, random_state=1)

In [40]:
print(f"Train size on {len(train_x)} days, while Test Size on {len(test_x)} day")

Train size on 69 days, while Test Size on 24 day


In [41]:
test_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6)

In [42]:
test_model.fit(train_x, train_y)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [43]:
predictions = test_model.predict(test_x)

In [44]:
predictions

array([844.24725, 745.9097 , 840.7999 , 861.68964, 719.1534 , 769.0299 ,
       833.2549 , 808.5979 , 864.68506, 783.55023, 852.82666, 822.27045,
       819.0204 , 792.3177 , 808.4776 , 864.42676, 839.64374, 860.1632 ,
       805.68835, 789.1963 , 803.34845, 818.09686, 805.8876 , 811.6881 ],
      dtype=float32)

In [45]:
nov_dec_test = pd.DataFrame({
    "actual": test_y.orders,
    "predicted": predictions,
    "dow": test_x['talabat_week'].values
}, index=test_y.index)

In [46]:
nov_dec_test

,actual,predicted,dow
119,843,844.247253,6
56,736,745.909729,0
59,848,840.799927,5
107,841,861.689636,3
112,779,719.153381,0
68,784,769.029907,3
8,780,833.254883,2
95,838,808.597900,0
99,790,864.685059,1
39,786,783.550232,0


In [47]:
# Getting Mean Absolute Error and Mean Absolute Percentage Error
mae = mean_absolute_error(test_y, predictions)
MAPE = mean_absolute_percentage_error(test_y, predictions) * 100
print(f"MAE = {mae:.2f} Orders")
print(f"MAPE = {MAPE}%")

MAE = 29.56 Orders
MAPE = 3.6800548434257507%


In [48]:
# Since any model cannot take datetime dtype so we're going to convert it to a numeric feature
test_daily_orders["order_year"] = daily_orders["order_date"].dt.year
test_daily_orders["order_month"] = daily_orders["order_date"].dt.month
test_daily_orders["order_day"] = daily_orders["order_date"].dt.day
test_daily_orders["dayofyear"] = daily_orders["order_date"].dt.dayofyear
test_daily_orders["quarter"] = daily_orders["order_date"].dt.quarter

In [49]:
test_daily_orders = test_daily_orders.drop('quarter', axis=1)
test_daily_orders

,order_date,orders,order_year,order_month,order_day,dayofyear
0,01/01/2022,769,2022,1,1,1
1,01/09/2021,819,2021,9,1,244
2,01/10/2021,837,2021,10,1,274
3,01/11/2021,765,2021,11,1,305
4,01/12/2021,764,2021,12,1,335
...,...,...,...,...,...,...
119,30/10/2021,843,2021,10,30,303
120,30/11/2021,795,2021,11,30,334
121,30/12/2021,780,2021,12,30,364
122,31/10/2021,857,2021,10,31,304


In [50]:
test_daily_orders = test_daily_orders.drop('order_date', axis=1)
test_daily_orders

,orders,order_year,order_month,order_day,dayofyear
0,769,2022,1,1,1
1,819,2021,9,1,244
2,837,2021,10,1,274
3,765,2021,11,1,305
4,764,2021,12,1,335
...,...,...,...,...,...
119,843,2021,10,30,303
120,795,2021,11,30,334
121,780,2021,12,30,364
122,857,2021,10,31,304


In [51]:
# Prepare X, y
X = test_daily_orders[["order_day", "order_month", "order_year"]]
y = test_daily_orders[["orders"]]

In [52]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [53]:
print(f"Train size on {len(x_train)} days, while Test Size on {len(x_test)} day")

Train size on 99 days, while Test Size on 25 day


In [54]:
# Since the model's error is relatively low, we can use it on all the train data and the above was a test on a small set of the df
model = XGBRegressor(n_estimators=20, max_depth=12, learning_rate=0.1, random_state=42, subsample=0.5)

In [55]:
model.fit(x_train, y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [56]:
prediction = model.predict(x_test)

In [57]:
prediction

array([798.579  , 860.1427 , 786.0391 , 856.3981 , 766.8424 , 817.84143,
       803.0987 , 851.8929 , 848.8554 , 786.99084, 832.5295 , 771.54297,
       803.0164 , 780.72626, 860.2114 , 800.19556, 786.2049 , 779.11523,
       860.1427 , 828.0362 , 789.96844, 834.384  , 745.8381 , 778.0254 ,
       772.3341 ], dtype=float32)

In [58]:
y_test.orders.dtype

dtype('int64')

In [59]:
results = pd.DataFrame({
    "actual": y_test.orders,
    "predicted": prediction
}, index=y_test.index)

In [60]:
# Calculate Error metrics
mae = mean_absolute_error(results['actual'], results['predicted'])
mape = mean_absolute_percentage_error(y_true=results['actual'], y_pred=results['predicted']) * 100

In [61]:
print(f"Model Performance:")
print(f"Mean Absolute Error: {mae:.2f} orders")
print(f"Mean Absolute Percentage Error: {mape:.2f}%")

Model Performance:
Mean Absolute Error: 40.16 orders
Mean Absolute Percentage Error: 4.89%


#### Using Prophet Model

- Prophet is a forecasting procedure designed for business time series data developed by Facebook's Core Data Science team.

In [62]:
from prophet import Prophet

In [63]:
daily_orders = daily_orders.drop('talabat_week', axis=1)
daily_orders = daily_orders.rename(columns={"order_date": "ds", "orders":"y"})
daily_orders

,ds,y
1,2021-09-01,819
6,2021-09-02,856
10,2021-09-03,837
14,2021-09-04,845
18,2021-09-05,847
...,...,...
117,2021-12-29,840
121,2021-12-30,780
123,2021-12-31,822
0,2022-01-01,769


In [64]:
p_model = Prophet(weekly_seasonality=True, yearly_seasonality=False)

In [65]:
p_model.fit(daily_orders)

16:18:46 - cmdstanpy - INFO - Chain [1] start processing


16:18:47 - cmdstanpy - INFO - Chain [1] done processing


In [66]:
jan_feb = p_model.make_future_dataframe(periods=59, freq="D")
jan_feb = jan_feb[jan_feb['ds'].dt.year.isin([2022])]
jan_feb = jan_feb[jan_feb['ds'].dt.month.isin([1,2])]

In [67]:
forecast = p_model.predict(jan_feb)

In [68]:
forecast.columns

Index(['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper',
       'additive_terms', 'additive_terms_lower', 'additive_terms_upper',
       'weekly', 'weekly_lower', 'weekly_upper', 'multiplicative_terms',
       'multiplicative_terms_lower', 'multiplicative_terms_upper', 'yhat'],
      dtype='object')

In [69]:
results = forecast[['ds', 'yhat_lower', 'yhat_upper', 'yhat']]
results

,ds,yhat_lower,yhat_upper,yhat
0,2022-01-01,715.412462,802.291993,761.294935
1,2022-01-02,697.672040,791.776216,744.841850
2,2022-01-03,697.107613,789.199879,743.426041
3,2022-01-04,727.303866,819.934394,774.583903
4,2022-01-05,721.387465,813.190353,768.124606
5,2022-01-06,744.460565,833.972682,787.889819
6,2022-01-07,761.086946,854.738821,807.877152
7,2022-01-08,711.853355,805.640790,757.255775
8,2022-01-09,692.592228,783.881901,740.802690
9,2022-01-10,692.339265,787.159648,739.386881


In [70]:
results.to_csv("forecast_prediction.csv", index=False)